In [ ]:
!pip install dummyPy
#!git clone https://github.com/neonbjb/ml-notebooks.git

# from google.colab import drive
# drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import csv
from math import ceil
import pandas as pd
import string
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, LSTM, Dense, LSTMCell
from keras.preprocessing.text import one_hot
from keras.utils import Sequence
from keras.utils import plot_model
from dummyPy import OneHotEncoder
from scipy import spatial
from sys import setrecursionlimit

In [ ]:
basedir = "MRS/"
#songlistdf = pd.read_csv(os.path.join(basedir,'spotify_songs.csv'))
playlistdf = pd.read_csv(os.path.join(basedir,'Dataset/spotify_data.csv'))

NUM_SAMPLES = 1000
train_samples = 600
val_samples = 200
test_samples = 200

#vocab_len = len(songlistdf)

In [ ]:
max_seq_len = 0
playlistdf['pid'].apply(int)

for name,group in playlistdf.groupby('pid'):
  max_seq_len = max(max_seq_len,len(group))

onehotencoder = OneHotEncoder(["time_signature", "key", "mode"])
onehotencoder.fit(playlistdf)
proc_playlistdf=onehotencoder.transform(playlistdf)
cols_to_norm = ['duration_ms','loudness','tempo']
proc_playlistdf[cols_to_norm] = proc_playlistdf[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
print(max_seq_len)
print(proc_playlistdf.shape)
print(proc_playlistdf.columns.values)

245
(67500, 37)
['acousticness' 'analysis_url' 'artist_name' 'danceability' 'duration_ms'
 'energy' 'id' 'instrumentalness' 'key_0' 'key_1' 'key_2' 'key_3' 'key_4'
 'key_5' 'key_6' 'key_7' 'key_8' 'key_9' 'key_10' 'key_11' 'liveness'
 'loudness' 'mode_0' 'mode_1' 'pid' 'speechiness' 'tempo'
 'time_signature_0' 'time_signature_1' 'time_signature_3'
 'time_signature_4' 'time_signature_5' 'track_href' 'track_name' 'type'
 'uri' 'valence']


In [ ]:
reg_features = ['acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence']
key_features = ['key_0','key_1','key_2','key_3','key_4','key_5','key_6','key_7','key_8','key_9','key_10','key_11']
mode_features = ['mode_0','mode_1']
timesig_features= ['time_signature_0', 'time_signature_1', 'time_signature_3', 'time_signature_4', 'time_signature_5']

def get_data_arrays(groupbyuser):
  
  enc_input_data = []
  dec_input_data = []
  dec_target_reg = []
  dec_target_key = []
  dec_target_mode = []
  dec_target_timesig = []

  for name,group in groupbyuser:
      grp = group.drop(['analysis_url','artist_name','id','track_href','track_name','type','uri','pid'],axis=1)
      
      grouplist = grp.values
      grouplist_reg = grp[reg_features].values
      grouplist_key = grp[key_features].values
      grouplist_mode = grp[mode_features].values
      grouplist_timesig = grp[timesig_features].values
      
      xgroup = grouplist[0:len(grouplist)-1]
      ygroup = grouplist[1:len(grouplist)]
      ytarget_reg = grouplist_reg[2:len(grouplist_reg)]
      ytarget_key = grouplist_key[2:len(grouplist_key)]
      ytarget_mode = grouplist_mode[2:len(grouplist_mode)]
      ytarget_timesig = grouplist_timesig[2:len(grouplist_timesig)]
      
      for i in range((max_seq_len-len(xgroup))):
          xgroup=np.vstack([xgroup,np.zeros(xgroup.shape[1])])

      for i in range((max_seq_len-len(ygroup))):
          ygroup=np.vstack([ygroup,np.zeros(ygroup.shape[1])])

      for i in range((max_seq_len-len(ytarget_reg))):
          ytarget_reg=np.vstack([ytarget_reg,np.zeros(ytarget_reg.shape[1])])
      
      for i in range((max_seq_len-len(ytarget_key))):
          ytarget_key=np.vstack([ytarget_key,np.zeros(ytarget_key.shape[1])])
      
      for i in range((max_seq_len-len(ytarget_mode))):
          ytarget_mode=np.vstack([ytarget_mode,np.zeros(ytarget_mode.shape[1])])
      
      for i in range((max_seq_len-len(ytarget_timesig))):
          ytarget_timesig=np.vstack([ytarget_timesig,np.zeros(ytarget_timesig.shape[1])])

      enc_input_data.append(xgroup)
      dec_input_data.append(ygroup)
      dec_target_reg.append(ytarget_reg)
      dec_target_key.append(ytarget_key)
      dec_target_mode.append(ytarget_mode)
      dec_target_timesig.append(ytarget_timesig)

  enc_input_data= np.array(enc_input_data)
  dec_input_data = np.array(dec_input_data)
  dec_target_reg = np.array(dec_target_reg)
  dec_target_key = np.array(dec_target_key)
  dec_target_mode = np.array(dec_target_mode)
  dec_target_timesig = np.array(dec_target_timesig)
  
  return enc_input_data,dec_input_data,dec_target_reg,dec_target_key,dec_target_mode,dec_target_timesig




In [ ]:
groupbyuser = proc_playlistdf.groupby('pid')
enc_input,dec_input,dec_target_reg,dec_target_key,dec_target_mode,dec_target_timesig= get_data_arrays(groupbyuser)

In [ ]:
enc_input_train,dec_input_train,dec_target_reg_train,dec_target_key_train,dec_target_mode_train,dec_target_timesig_train = enc_input[:train_samples],dec_input[:train_samples],dec_target_reg[:train_samples],dec_target_key[:train_samples],dec_target_mode[:train_samples],dec_target_timesig[:train_samples]

enc_input_val,dec_input_val,dec_target_reg_val,dec_target_key_val,dec_target_mode_val,dec_target_timesig_val = enc_input[train_samples:train_samples + val_samples],dec_input[train_samples:train_samples+val_samples],dec_target_reg[train_samples:train_samples+val_samples],dec_target_key[train_samples:train_samples+val_samples],dec_target_mode[train_samples:train_samples+val_samples],dec_target_timesig[train_samples:train_samples+val_samples]

enc_input_test,dec_input_test,dec_target_reg_test,dec_target_key_test,dec_target_mode_test,dec_target_timesig_test = enc_input[train_samples + val_samples:train_samples + val_samples + test_samples],dec_input[train_samples + val_samples:train_samples + val_samples + test_samples],dec_target_reg[train_samples + val_samples:train_samples + val_samples + test_samples],dec_target_key[train_samples + val_samples:train_samples + val_samples + test_samples],dec_target_mode[train_samples + val_samples:train_samples + val_samples + test_samples],dec_target_timesig[train_samples + val_samples:train_samples + val_samples + test_samples]

print(enc_input_train.shape)
print(dec_input_train.shape)
print(dec_target_reg_train.shape)
print(dec_target_key_train.shape)
print(dec_target_mode_train.shape)
print(dec_target_timesig_train.shape)


(600, 245, 29)
(600, 245, 29)
(600, 245, 10)
(600, 245, 12)
(600, 245, 2)
(600, 245, 5)


In [ ]:
setrecursionlimit(10000)

song_vocab = proc_playlistdf.drop(['analysis_url','artist_name','track_href','type','uri','pid'],axis=1).drop_duplicates('id')

song_vocab_vecs = song_vocab.drop(['id','track_name'],axis=1).values

sim_tree = spatial.KDTree(song_vocab_vecs)

proc_playlistdf.to_csv(os.path.join(basedir,"Dataset/norm_playlists.csv"),index=False)
song_vocab.to_csv(os.path.join(basedir,"Dataset/norm_songs.csv"),index=False)


In [ ]:
# Generator function to generate 
class my_generator(Sequence):

    def __init__(self, enc_inp,dec_inp,dec_tar_reg,dec_tar_key,dec_tar_mode,dec_tar_timesig,batch_size):
        self.enc_inp = enc_inp
        self.dec_inp = dec_inp
        self.dec_tar_reg = dec_tar_reg
        self.dec_tar_key = dec_tar_key
        self.dec_tar_mode = dec_tar_mode
        self.dec_tar_timesig = dec_tar_timesig
        self.batch_size = batch_size

    def __len__(self):
        return ceil(self.enc_inp.shape[0] / float(self.batch_size))

    def __getitem__(self, idx):
        enc_inp_batch = self.enc_inp[idx*batch_size:(idx+1)*batch_size]
        dec_inp_batch = self.dec_inp[idx*batch_size:(idx+1)*batch_size]
        dec_tar_reg_batch = self.dec_tar_reg[idx*batch_size:(idx+1)*batch_size]
        dec_tar_key_batch = self.dec_tar_key[idx*batch_size:(idx+1)*batch_size]
        dec_tar_mode_batch = self.dec_tar_mode[idx*batch_size:(idx+1)*batch_size]
        dec_tar_timesig_batch = self.dec_tar_timesig[idx*batch_size:(idx+1)*batch_size]
        
        return [enc_inp_batch,dec_inp_batch],[dec_tar_reg_batch,dec_tar_key_batch,dec_tar_mode_batch,dec_tar_timesig_batch]


In [ ]:
# training model
latent_dim = 256

encoder_inputs = Input(shape=(None, enc_input.shape[2]))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, dec_input.shape[2]))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
  
reg_layer = Dense(dec_target_reg.shape[2],activation = 'sigmoid', name = 'reg_output')
key_layer = Dense(dec_target_key.shape[2],activation = 'softmax', name = 'key_output')
mode_layer = Dense(dec_target_mode.shape[2],activation = 'softmax', name = 'mode_output')
timesig_layer = Dense(dec_target_timesig.shape[2],activation = 'softmax', name = 'timesig_output')
  
decoder_outputs_reg = reg_layer(decoder_outputs)
decoder_outputs_key = key_layer(decoder_outputs)
decoder_outputs_mode = mode_layer(decoder_outputs)
decoder_outputs_timesig = timesig_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], [decoder_outputs_reg,decoder_outputs_key,decoder_outputs_mode,decoder_outputs_timesig])


In [ ]:
batch_size = 32
epochs = 20 

train_gen = my_generator(enc_input_train,dec_input_train,dec_target_reg_train,dec_target_key_train,dec_target_mode_train,dec_target_timesig_train,batch_size)
val_gen = my_generator(enc_input_val,dec_input_val,dec_target_reg_val,dec_target_key_val,dec_target_mode_val,dec_target_timesig_val,batch_size)

loss_funcs = {
    'reg_output':'mse',
    'key_output':'categorical_crossentropy',
    'mode_output':'categorical_crossentropy',
    'timesig_output':'categorical_crossentropy'
}

loss_wts = {
    'reg_output': 1.0,
    'key_output': 1.0,
    'mode_output': 1.0,
    'timesig_output': 1.0
}

model.compile(optimizer='rmsprop', loss=loss_funcs,loss_weights=loss_wts )

model.fit_generator(generator=train_gen,
                    validation_data=val_gen,
                    use_multiprocessing=True,
                    workers=4,
                    epochs=epochs)

Epoch 1/20
18/19 [===========================>..] - ETA: 0s - loss: 1.0524 - reg_output_loss: 0.1029 - key_output_loss: 0.6504 - mode_output_loss: 0.1849 - timesig_output_loss: 0.1141Epoch 1/20
Epoch 1/20
19/19 [==============================] - 19s 990ms/step - loss: 1.0379 - reg_output_loss: 0.0990 - key_output_loss: 0.6447 - mode_output_loss: 0.1828 - timesig_output_loss: 0.1114 - val_loss: 0.9230 - val_reg_output_loss: 0.0294 - val_key_output_loss: 0.6465 - val_mode_output_loss: 0.1687 - val_timesig_output_loss: 0.0785
Epoch 2/20
19/19 [==============================] - 15s 780ms/step - loss: 0.9167 - reg_output_loss: 0.0213 - key_output_loss: 0.6415 - mode_output_loss: 0.1715 - timesig_output_loss: 0.0824 - val_loss: 0.9045 - val_reg_output_loss: 0.0125 - val_key_output_loss: 0.6454 - val_mode_output_loss: 0.1699 - val_timesig_output_loss: 0.0768
Epoch 3/20
19/19 [==============================] - 15s 796ms/step - loss: 0.9098 - reg_output_loss: 0.0198 - key_output_loss: 0.6404 - 

In [ ]:
#inferencing model

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

decoder_outputs_reg = reg_layer(decoder_outputs)
decoder_outputs_key = key_layer(decoder_outputs)
decoder_outputs_mode = mode_layer(decoder_outputs)
decoder_outputs_timesig = timesig_layer(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs_reg,decoder_outputs_key,decoder_outputs_mode,decoder_outputs_timesig] + decoder_states)

In [ ]:
def prep_sos_seq(input_seq):

    inp_seq = input_seq[0]
    reg_idx = [0,1,2,3,4,17,18,21,22,28]
    key_idx = [5,6,7,8,9,10,11,12,13,14,15,16]
    mode_idx = [19,20]
    time_sig_idx = [23,24,25,26,27]

    inp_seq_reg = np.mean(np.take(inp_seq,reg_idx,axis=1),axis=0)

    inp_seq_key = np.zeros(12)
    inp_seq_key[np.argmax(np.sum(np.take(inp_seq,key_idx,axis=1),axis=0))] = 1

    inp_seq_mode = np.zeros(2)
    inp_seq_key[np.argmax(np.sum(np.take(inp_seq,mode_idx,axis=1),axis=0))] = 1

    inp_seq_timesig = np.zeros(5)
    inp_seq_timesig[np.argmax(np.sum(np.take(inp_seq,time_sig_idx,axis=1),axis=0))] = 1

    sos_vec = np.concatenate((inp_seq_reg[0:5],inp_seq_key))
    sos_vec = np.concatenate((sos_vec,inp_seq_reg[5:7]))
    sos_vec = np.concatenate((sos_vec,inp_seq_mode))
    sos_vec = np.concatenate((sos_vec,inp_seq_reg[7:9]))
    sos_vec = np.concatenate((sos_vec,inp_seq_timesig))
    sos_vec = np.append(sos_vec,inp_seq_reg[9])

    sos_vec = sos_vec.reshape((1,1,-1))

In [ ]:
max_playlist_len = 5

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    sos_vec = prep_sos_seq(input_seq)
    
    eos_vec = np.zeros(29)
    
    target_seq = sos_vec
    
    stop_condition = False
    rec_song_ids = []
    
    while not stop_condition:
      
        reg, key, mode, timesig ,h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Process outputs to produce input for next time step
        songid,song_vec = getclosestsong(reg[0,-1,:],key[0,-1,:],mode[0,-1,:],timesig[0,-1,:])
        rec_song_ids.append(songid)
      
        # Exit condition:  hit max sequence length
        if (len(rec_song_ids) > max_playlist_len):
            stop_condition = True
        
        # Update the target sequence (of length 1).
        target_seq = song_vec

        # Update states
        states_value = [h, c]

    return rec_song_ids

In [ ]:
def getclosestsong(reg,key,mode,timesig):
  feat_vec_pred = {
      'acousticness':reg[0],
      'danceability':reg[1],
      'duration_ms':reg[2],
      'energy':reg[3],
      'instrumentalness':reg[4],
      'key_0':key[0],
      'key_1':key[1],
      'key_2':key[2],
      'key_3':key[3],
      'key_4':key[4],
      'key_5':key[5],
      'key_6':key[6],
      'key_7':key[7],
      'key_8':key[8],
      'key_9':key[9],
      'key_10':key[10],
      'key_11':key[11],
      'liveness':reg[5],
      'loudness':reg[6],
      'mode_0':mode[0],
      'mode_1':mode[1],
      'speechiness':reg[7],
      'tempo':reg[8],
      'time_signature_0':timesig[0],
      'time_signature_1':timesig[1],
      'time_signature_3':timesig[2],
      'time_signature_4':timesig[3],
      'time_signature_5':timesig[4],
      'valence':reg[9]
  }
  
  feat_vec_pred = np.array(list(feat_vec_pred.values()))
  
  _ , idx = sim_tree.query(feat_vec_pred)
  
  songid =  song_vocab.iloc[idx]['id']
  
  songvec = song_vocab_vecs[idx].reshape((1,1,-1))
  
  return songid,songvec

In [ ]:
ipseqtest = np.array([enc_input_test[16]])

recommendation_ids = decode_sequence(ipseqtest)

print(recommendation_ids)

AttributeError: ignored

In [ ]:
model.save(os.path.join(basedir,"Model/train_model.h5"))
encoder_model.save(os.path.join(basedir,"Model/inf_encoder.h5"))
decoder_model.save(os.path.join(basedir,"Model/inf_decoder.h5"))

plot_model(model,to_file = os.path.join(basedir,'Model/model.png'))
plot_model(encoder_model,to_file = os.path.join(basedir,'Model/encoder.png'))
plot_model(decoder_model,to_file = os.path.join(basedir,'Model/decoder.png'))

In [ ]:
np.take([[13,4,1],[2,5,6],[7,0,11]],[0,1],axis=1)